#Imports 

In [14]:
!pip install pytorch_lightning
!pip install lightning-bolts 
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions import Beta
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy
import pandas as pd
from pytorch_lightning.callbacks import ModelCheckpoint
from  pl_bolts import optimizers
import copy

from pytorch_lightning import seed_everything

#Data Download

In [15]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
!gunzip  covtype.data.gz

--2021-08-20 04:02:27--  https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11240707 (11M) [application/x-httpd-php]
Saving to: ‘covtype.data.gz’

covtype.data.gz     100%[===================>]  10.72M  65.5MB/s    in 0.2s    

2021-08-20 04:02:27 (65.5 MB/s) - ‘covtype.data.gz’ saved [11240707/11240707]

gzip: covtype.data already exists; do you wish to overwrite (y or n)? n
	not overwritten


#Utils

In [16]:
class Mixup:
  def __init__(self,alpha):
    '''
    mixup the data with mixing coeffecient
    '''
    super().__init__()
    self.alpha=alpha
  def __call__(self,x):
    lam=Beta(self.alpha,self.alpha).sample()
    randidx = torch.randperm(len(x)).to(x.device)
    x = lam * x + (1-lam) * x[randidx]
    return x, lam , randidx


In [17]:
class Maxout(nn.Module):
  '''
  Apply a maxout pooling layer
  '''
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[-1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[-1], self._pool_size)
        m, i = x.view(*x.shape[:-1], x.shape[-1] // self._pool_size, self._pool_size).max(-1)
        return m

#Model architecture

In [18]:
class TabulerModel(nn.Module):
  '''
  5 layer Mlp with Projection Head and Maxout layer
  '''
    def __init__(self,hid_dim,input_dim,head_dim,num_classes,pool_size):
        super(TabulerModel, self).__init__()
        
        self.layer = nn.Sequential(nn.Linear(input_dim,hid_dim),
                    nn.BatchNorm1d(hid_dim),
                    nn.ReLU(),
                    nn.Linear(hid_dim,hid_dim),
                    nn.BatchNorm1d(hid_dim),
                    nn.ReLU(),
                    nn.Linear(hid_dim,hid_dim*2),
                    nn.BatchNorm1d(hid_dim*2),
                    nn.ReLU(),
                    nn.Linear(hid_dim*2,hid_dim*2),
                    nn.BatchNorm1d(hid_dim*2),
                    nn.ReLU(),
                    nn.Linear(hid_dim*2,hid_dim*2*2),
                    nn.BatchNorm1d(hid_dim*2*2)) 
        self.maxout=Maxout(pool_size)
        self.projectHead = nn.Sequential(nn.Linear(hid_dim,hid_dim),
                                         nn.ReLU(),
                                         nn.Linear(hid_dim,head_dim))
        
       
        
    def forward(self, inputs,):
        x = self.layer(inputs)
        x_max = self.maxout(x)
        
        return self.projectHead(x_max)



#Dataset class

In [20]:
class DatasetTabular(Dataset):
  '''
  Tabular data class
  '''
  def __init__(self, data, y ):
    super().__init__()
    self.data=data.values
    self.y=y.values
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    return self.data[idx],self.y[idx]


def create_dataset(path):
  '''
  create dataset object from path
  '''
  df=pd.read_csv(path,header=None)
  df.iloc[:,-1]=df.iloc[:,-1]-1
  df.iloc[:,:11]=(df.iloc[:,:11]-df.iloc[:,:11].mean())/df.iloc[:,:11].std()
  x,y=df.iloc[:,:-1],df.iloc[:,-1]
  dataset=DatasetTabular(x,y)
  return dataset

def generate_splits(dataset, split):
  '''
  split data 
  '''
  train_sz, test_sz=len(dataset)-int(len(dataset)*split),int(len(dataset)*split)
  train,test=torch.utils.data.random_split(dataset,[train_sz ,test_sz])
  return train, test


def tabularaugment(input,prob=0.2):
  '''
  Data Augmentation adding noise
  '''
  matrix=torch.ones(input.shape)*prob
  
  p=torch.bernoulli(matrix).to(input.device)
 
  output =input+p*torch.randn(input.shape,device=input.device)

  return output 



In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Trainer-Supervised 

In [22]:

class TestSUP(pl.LightningModule):
  '''
  create a fine tuned model
  '''
 
  def __init__(self,model,fc,learningRate,weight_decay, momentum,num_output,freeze_encoder=True,layers_to_freeze=6):
     super().__init__()
     self.model=model
     self.fc=fc
     self.learningRate=learningRate
     self.weight_decay=weight_decay
     self.momentum=momentum
     self.setup_criterion()
     self.train_acc=Accuracy(num_classes=num_output)
     self.validation_acc=Accuracy(num_classes=num_output)
     self.test_acc=Accuracy(num_classes=num_output)
     if freeze_encoder:
       self.model.eval()
       ct=0
       for child in self.model.children():
            ct += 1
            if ct < layers_to_freeze:
                for param in child.parameters():
                    param.requires_grad = False
       #self.model.projectHead=fc
  def forward(self,x):
    return self.model(x)
     

  def _shared_step(self, batch, batch_idx,accuracy):
    x,y=batch
    pretrain=self.model(x.float())
   
    #output=self.fc(pretrain)
    prediction=torch.sigmoid(pretrain)
    loss=self.criterion(pretrain.squeeze(),y)
  
    accuracy.update(prediction,y)
    return loss 

  def training_step(self, batch,batch_idx):
    
    loss=self._shared_step(batch,batch_idx,self.train_acc)
    self.log('train_loss', loss, on_step=True, 
                 on_epoch=True, prog_bar=True, logger=True)
  
    return {'loss': loss}
  def train_epoch_end(self, out): 
    self.log('training accuracy',self.train_acc.compute(),prog_bar=True,)
    self.train_acc.reset()
  def validation_step(self, batch,batch_idx):
    loss=self._shared_step(batch,batch_idx,self.validation_acc)
 
    self.log('val_loss', loss, on_step=True, 
                 on_epoch=True, prog_bar=True, logger=True)
  
 
  def validation_epoch_end(self, out):
    self.log('validation accuracy',self.validation_acc.compute(),prog_bar=True,)
    print(self.validation_acc.compute())
    self.validation_acc.reset()
  
  def test_step(self,batch,batch_idx):
     loss=self._shared_step(batch,batch_idx,self.test_acc)
     self.log('test_loss', loss, on_step=True, 
                 on_epoch=True, prog_bar=True, logger=True)
  def test_epoch_end(self, out): 
    self.log('test accuracy',self.test_acc.compute(),prog_bar=True,)
    self.test_acc.reset()

  def configure_optimizers(self): 
    optimizer =torch.optim.SGD(self.parameters(), lr=self.learningRate)
    scheduler=optimizers.lr_scheduler.LinearWarmupCosineAnnealingLR(optimizer,warmup_epochs=10, max_epochs=500)
    return {
                'optimizer': optimizer,
                "lr_scheduler": scheduler
            }

  def setup_criterion(self):
    self.criterion= nn.CrossEntropyLoss()

 


#Trainer-Self supervised(N-Pair)

In [23]:
class Npair(pl.LightningModule):
  '''
  pretrain an N-Pair model with i-mix Loss
  '''
 
  def __init__(self,model,alpha, tabularaugment,t,learningRate,weight_decay, momentum, use_imix=False):
     super().__init__()
     self.model=model
     self.alpha=alpha
     self.mixup=Mixup(alpha)
     self.augment=tabularaugment
     self.t=t
     self.learningRate=learningRate
     self.weight_decay=weight_decay
     self.momentum=momentum
     self.setup_criterion()
     self.use_imix=use_imix
     
  def _shared_step(self,batch,batch_idx):
    x,_=batch
    
    r= self.augment(x.float())
    
    r_prime=self.augment(x.float())
    #calculating the loss
    if self.use_imix:
      r_mix , lam , randidx=self.mixup(r)

      randidx = randidx.to(self.device)
      logits = torch.matmul(F.normalize(model(r_mix)), F.normalize(model(r_prime)).T) / self.t
      loss= lam * self.criterion(logits, torch.arange(len(x)).to(self.device)) + \
        (1-lam) * self.criterion(logits, randidx)
     
    else:
      logits=torch.matmul(F.normalize(model(r)), F.normalize(model(r_prime)).T) / self.t
      loss=self.criterion(logits, torch.arange(len(x)).to(x.device))
      
    return loss


  def training_step(self, batch,batch_idx):
    loss=self._shared_step(batch,'train')
    print(loss)
    self.log('loss', loss, on_step=True, 
                 on_epoch=True, prog_bar=True, logger=True)
    return {'loss': loss}
  def validation_step(self,batch,batch_idx):
    loss=self._shared_step(batch,'val')
 
    self.log('val_loss', loss, on_step=True, 
                 on_epoch=True, prog_bar=True, logger=True)
    
    return {'val_loss': loss}
  def configure_optimizers(self): 
    optimizer =torch.optim.SGD(self.parameters(), lr=self.learningRate,weight_decay=self.weight_decay,momentum=self.momentum)
    scheduler=optimizers.lr_scheduler.LinearWarmupCosineAnnealingLR(optimizer,warmup_epochs=10, max_epochs=5000)
    return {
                'optimizer': optimizer,
                "lr_scheduler": scheduler
            }



  def setup_criterion(self):
    self.criterion= nn.CrossEntropyLoss()


#Setup Experiment 

In [24]:
#pretraining dataset
dataset=create_dataset(path='/content/covtype.data')
train, test=generate_splits(dataset,split=0.99)
#create pretrain dataloader
pin_memory = True if torch.cuda.is_available() else False
train_dataloader=DataLoader(train,batch_size=512,pin_memory=pin_memory)
test_dataloader=DataLoader(test,batch_size=512,pin_memory=pin_memory)
#create finetune dataloader
trainF, valF=generate_splits(train,split=0.2)
trainF_dataloader=DataLoader(trainF,batch_size=512,pin_memory=pin_memory)
valF_dataloader=DataLoader(valF,batch_size=512,pin_memory=pin_memory)
#create model
model=TabulerModel(hid_dim=2048,input_dim=54,head_dim=128,num_classes=7,pool_size=4)

In [25]:
pretrained_checkpoint='/content/drive/MyDrive/checkpoints6/i-Mix-NpairFineTuneepoch=341-loss=2.67.ckpt'
experiment='pre-train'
hid_dim=2048
num_classes=7
seed_everything(123)
if experiment=='pre-train':
    LitModel=Npair(model,alpha=2,tabularaugment=tabularaugment,t=0.1,
                       learningRate=0.125,weight_decay=0.0001,momentum=0.9,use_imix=True)
elif experiment=='fine-tune':
    fc=nn.Linear(hid_dim,num_classes)
    LitModel=TestSUP(model,fc=fc,learningRate=0.001,
                     weight_decay=0,momentum=0.9,num_output=num_classes,freeze_encoder=True)
    if pretrained_checkpoint:
      unsup_param=dict(model=model,alpha=2,
                       tabularaugment=tabularaugment,t=0.1,
                       learningRate=0.125*2
                      ,weight_decay=0.001,momentum=0.9)
      pretrained_Model=Npair(**unsup_param)
      pretrained_Model.load_from_checkpoint(checkpoint_path=pretrained_checkpoint,**unsup_param,strict=False)
      LitModel.model=copy.deepcopy(pretrained_Model.model)
      LitModel.model.projectHead= fc



Global seed set to 123


In [26]:
#Trainign
checkpoint_callback = ModelCheckpoint(monitor="loss",mode='min',
                                      filename="i-Mix-pretrain{epoch:02d}-{val-loss:.2f}",
                                      dirpath='/content/drive/MyDrive/checkpoints8',
                                      verbose=True,
                                      save_last=True)

In [ ]:

logger = TensorBoardLogger("tb4_logs", name="LitModel")
trainer=pl.Trainer(max_epochs=400, gpus=-1,auto_select_gpus=True,
                   callbacks=[checkpoint_callback],
                   #default_root_dir='/content/checkpoints',
                   logger=logger)
if experiment=='pre-train':
    trainer.fit(LitModel,train_dataloader)
    checkpoint=checkpoint_callback.best_model_path
else:
    trainer.fit(LitModel,trainF_dataloader,valF_dataloader)

#score= checkpoint_callback.best_model_score


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  rank_zero_warn(f"you defined a {step_name} but have no {loader_name}. Skipping {stage} loop")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type         | Params
---------------------------------------
0 | model | TabulerModel | 67.5 M
---------------------------------------
67.5 M    Trainable params
0         Non-trainable params
67.5 M    Total params
270.181   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 123
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: -1it [00:00, ?it/s]

tensor(6.0793, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0425, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.9613, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.1033, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0764, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0942, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0958, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.9703, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.9540, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0576, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0642, device='cuda:0', grad_fn=<AddBackward0>)


...

tensor(3.8666, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7496, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9037, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.5499, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.4837, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0446, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7711, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.8211, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7252, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.3913, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3873, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.5082, device='cuda:0', grad_fn=<AddBackward0>)


Epoch 271, global step 3263: loss was not in top 1


tensor(3.4017, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9960, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.8576, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0680, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.4086, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7407, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2262, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7593, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2185, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1777, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5932, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0065, device='cuda:0', grad_fn=<AddBackward0>)


Epoch 272, global step 3275: loss was not in top 1


tensor(3.8510, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0498, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.3959, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7930, device='cuda:0', grad_fn=<AddBackward0>)


#Test

In [ ]:
trainer.test(ckpt_path='best',
                    test_dataloaders=test_dataloader,
                    )


#Tensor Board

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir ./tb_logs/

#Configs 

In [ ]:
from argparse import Namespace
args=Namespace(
    # experiment parameters
    experiment = 'pre-train', # {'pre-train', 'fine-tune'}

    pretrained_checkpoint = None, #'/content/drive/MyDrive/checkpoints6/i-Mix-NpairFineTuneepoch=341-loss=2.67.ckpt',
    

    #path to data
    path='/content/covtype.data',

    #Parameters for the model 
    hid_dim=2048,
    input_dim=54,
    head_dim=128,
    num_classes=7,
    pool_size=4,
    
    #Parameters for mixup
    alpha=2,

    # parameter for contrastive loss
    t = 0.2,


    # parameters for training
    batch_size=512,
    learningRate = 0.125,
    weight_decay = 0.0001,
    momentum=0.99,
    freeze_encoder = True, # freeze transformer layer

    num_epochs = 400
    no_of_gpus = -1,
    seed = 123,
    resume_checkpoint = None,
    monitor = 'val_loss', # {val_loss, val_auroc_epoch}
    monitor_mode = 'min' # {max, min}
    resume_checkpoint = None

   
)